In [20]:
!mkdir train
!mkdir val
!aws s3 cp s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/system/action-data/action_val.csv val/
!aws s3 cp s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/system/action-data/action_train.csv train/

mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘val’: File exists
download: s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/system/action-data/action_val.csv to val/action_val.csv
download: s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/system/action-data/action_train.csv to train/action_train.csv


In [21]:
from __future__ import print_function
import os
import sys
import math
import pickle
import boto3
import os
import numpy as np
import pandas as pd
# from tqdm import tqdm
import time
import argparse
import logging
import re
# tqdm.pandas()
# pandarallel.initialize(progress_bar=True)
# bucket = os.environ.get("BUCKET_NAME", " ")
# raw_data_folder = os.environ.get("RAW_DATA", " ")
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# tqdm_notebook().pandas()
s3client = boto3.client('s3')

In [41]:
########################################
# 从s3同步数据
########################################


def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    print("upload s3://{}/{}".format(bucket, key))
    with open(filename, 'rb') as f:  # Read in binary mode
        # return s3client.upload_fileobj(f, bucket, key)
        return s3client.put_object(
            ACL='bucket-owner-full-control',
            Bucket=bucket,
            Key=key,
            Body=f
        )

def write_str_to_s3(content, bucket, key):
    print("write s3://{}/{}, content={}".format(bucket, key, content))
    s3client.put_object(Body=str(content).encode("utf8"), Bucket=bucket, Key=key, ACL='bucket-owner-full-control')

default_bucket = 'aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887'
default_prefix = 'sample-data'
parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str, default=default_bucket)
parser.add_argument('--prefix', type=str, default=default_prefix)
args, _ = parser.parse_known_args()
bucket = args.bucket
prefix = args.prefix

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

out_s3_path = "s3://{}/{}/feature/content/inverted-list".format(bucket, prefix)

local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# dkn模型文件下载

file_name_list = ['dkn_entity_embedding.npy','dkn_context_embedding.npy','dkn_word_embedding.npy']

s3_folder = '{}/model/rank/content/dkn_embedding_latest/'.format(prefix)

sync_s3(file_name_list, s3_folder, local_folder)
                
!mv info/dkn_entity_embedding.npy train/entity_embeddings_TransE_128.npy
!mv info/dkn_context_embedding.npy train/context_embeddings_TransE_128.npy
!mv info/dkn_word_embedding.npy train/word_embeddings_300.npy

bucket=aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887
prefix='sample-data'
file preparation: download src key sample-data/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy to dst key info/dkn_entity_embedding.npy
file preparation: download src key sample-data/model/rank/content/dkn_embedding_latest/dkn_context_embedding.npy to dst key info/dkn_context_embedding.npy
file preparation: download src key sample-data/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy to dst key info/dkn_word_embedding.npy


In [43]:
!python embed_dkn.py --learning_rate 0.0001 --loss_weight 1.0 --max_click_history 16 --num_epochs 1 --use_entity True --use_context 0 --max_title_length 16 --entity_dim 128 --word_dim 300 --batch_size 128 --perform_shuffle 1 --data_dir /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn --checkpointPath /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/temp/ --servable_model_dir /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/model_complete/ 


check input params: 
['embed_dkn.py', '--learning_rate', '0.0001', '--loss_weight', '1.0', '--max_click_history', '16', '--num_epochs', '1', '--use_entity', 'True', '--use_context', '0', '--max_title_length', '16', '--entity_dim', '128', '--word_dim', '300', '--batch_size', '128', '--perform_shuffle', '1', '--data_dir', '/home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn', '--checkpointPath', '/home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/temp/', '--servable_model_dir', '/home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/model_complete/']
task_type  train
model_dir  
checkpoint_dir  /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/temp/
data_dir  /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn
dt_dir  20210420
num_epochs  1
batch_size  128
loss_type  log_loss
optimizer  Adam
learning_rate  0.0001
e

INFO:tensorflow:Saving checkpoints for 0 into /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/temp/model.ckpt.
I0421 07:42:36.480328 140143676024640 basic_session_run_hooks.py:606] Saving checkpoints for 0 into /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/temp/model.ckpt.
INFO:tensorflow:loss = 0.7773521, step = 0
I0421 07:42:38.420113 140143676024640 basic_session_run_hooks.py:262] loss = 0.7773521, step = 0
INFO:tensorflow:global_step/sec: 12.3371
I0421 07:42:46.524991 140143676024640 basic_session_run_hooks.py:692] global_step/sec: 12.3371
INFO:tensorflow:loss = 0.68353057, step = 100 (8.106 sec)
I0421 07:42:46.526058 140143676024640 basic_session_run_hooks.py:260] loss = 0.68353057, step = 100 (8.106 sec)
INFO:tensorflow:Saving checkpoints for 118 into /home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-dkn/temp/model.ckpt.
I0421 07:42:47.853429 140143676024640 basic_sess

In [10]:
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_context_embedding.npy s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/dkn_context_embedding.npy --acl bucket-owner-full-control
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy --acl bucket-owner-full-control
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy --acl bucket-owner-full-control
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/action/dkn/latest/model.tar.gz s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/action/dkn/latest/model.tar.gz --acl bucket-owner-full-control

copy: s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_context_embedding.npy to s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/dkn_context_embedding.npy
copy: s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy to s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy
copy: s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy to s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy
copy: s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/action/dkn/latest/model.tar.gz to s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/action/dkn/latest/model.tar.gz


In [11]:
# n = 0
# for k, v in sort_id_score_dict.items():
#     print("k {} v {}".format(k,v))
#     if n > 10:
#         break
#     n = n + 1

k 6552400705523548680 v 0.05671838
k 6552341601450983944 v 0.05580449
k 6552015749588713997 v 0.052325606
k 6552459085898318343 v 0.05156085
k 6551717707065065987 v 0.05155331
k 6552288949518205448 v 0.051062495
k 6552426608496476685 v 0.049799293
k 6552386898592006670 v 0.049290717
k 6552496722042421773 v 0.04928559
k 6525341647792767502 v 0.04853931
k 6552207083696030216 v 0.048426867
k 6552428222233969165 v 0.048256338
